In [1]:
import os
import pickle
import numpy as np

import torch
import torch.nn.functional
from torchsummary import summary

from pyeer.eer_info import get_eer_stats
from pyeer.report import generate_eer_report, export_error_rates
from pyeer.plot import plot_eer_stats

from config import models_folder, output_data_folder, EER_folder

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
output_folder = os.path.join(EER_folder, "mobilenetv2_vs_densenet121")

In [4]:
# Model input config
parent_dir = os.path.join(models_folder, "verification_classifier", "good_models")
model_folders = [
    "2020-03-20_12-20-57_mobilenet",
    "2020-04-06_16-34-21_densenet",
    "2020-04-10_13-13-49_var_red_mobilenet",
    "2020-04-10_15-21-25_var_red_densenet",
]
model_folders = [os.path.join(parent_dir, folder) for folder in model_folders]

In [5]:
# EER config
EER_model_names = [
    "MobileNetV2", "DenseNet121",
    "MobileNetV2_VR", "DenseNet121_VR",
]

In [6]:
# Output folder exists check
if os.path.exists(output_folder): raise FileExistsError("{} already exists".format(output_folder))
else: os.makedirs(output_folder)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.cuda.is_available()

cuda


True

In [8]:
def generate_model_scores(model_folder, validation_data):
    # Verification Binary classifier
    model = torch.jit.load(os.path.join(model_folder, "mobile_model.pt"))
    model.to(device)
    
    ### scores
    class_probs_all = []
    outputs_all = []
    labels_all = []
    geniune_scores_all = []
    impostor_scores_all = []

    for data in validation_data:
        # prep inputs
        input_imgs, labels = data
        inputs = [img.to(device) for img in input_imgs]
        # predict
        with torch.no_grad():
            model.eval()   # eval mode
            outputs = model(inputs)
            class_probs = torch.nn.functional.softmax(outputs, dim=1)

        # save model outputs
        # Convert to numpy
        class_probs = class_probs.cpu().detach().numpy()
        outputs = outputs.cpu().detach().numpy()
        labels = labels.cpu().detach().numpy()
        # append arrays
        class_probs_all.append(class_probs)
        outputs_all.append(outputs)
        labels_all.append(labels)
        # genuine/impostor scores
        genuine_idx = np.argwhere(labels == 1)
        imposter_idx = np.argwhere(labels == 0)
        # append values
        geniune_scores_all.extend(class_probs[genuine_idx, 1].flatten().tolist())
        impostor_scores_all.extend(class_probs[imposter_idx, 1].flatten().tolist())
    
    # Save scores
    scores_save_config = [
        (class_probs_all, "class_probs_all.pickle"),
        (outputs_all, "outputs_all.pickle"),
        (labels_all, "labels_all.pickle"),
        (geniune_scores_all, "geniune_scores_all.pickle"),
        (impostor_scores_all, "impostor_scores_all.pickle"),
    ]
    # Scores folder
    scores_folder = os.path.join(output_folder, os.path.basename(model_folder))
    if not os.path.exists(scores_folder): os.makedirs(scores_folder)
    for list_, filename in scores_save_config:
        file = os.path.join(scores_folder, filename)
        with open(file, 'wb') as handle:
            pickle.dump(list_, handle, protocol=pickle.HIGHEST_PROTOCOL)
        # Print count stats
        print("{}, Count: {}".format(filename, len(list_)))
        
    # For EER stats
    return geniune_scores_all, impostor_scores_all

In [9]:
# Validation data
validation_set_file = os.path.join(output_data_folder, "validation_sets", "verification_validation_set.pickle")
with open(validation_set_file, 'rb') as f:
    validation_data = pickle.load(f)

In [10]:
### Get scores
scores = []
for model_folder in model_folders:
    score = generate_model_scores(model_folder, validation_data)
    scores.append(score)

class_probs_all.pickle, Count: 12
outputs_all.pickle, Count: 12
labels_all.pickle, Count: 12
geniune_scores_all.pickle, Count: 1920
impostor_scores_all.pickle, Count: 1920
class_probs_all.pickle, Count: 12
outputs_all.pickle, Count: 12
labels_all.pickle, Count: 12
geniune_scores_all.pickle, Count: 1920
impostor_scores_all.pickle, Count: 1920
class_probs_all.pickle, Count: 12
outputs_all.pickle, Count: 12
labels_all.pickle, Count: 12
geniune_scores_all.pickle, Count: 1920
impostor_scores_all.pickle, Count: 1920
class_probs_all.pickle, Count: 12
outputs_all.pickle, Count: 12
labels_all.pickle, Count: 12
geniune_scores_all.pickle, Count: 1920
impostor_scores_all.pickle, Count: 1920


In [11]:
# EER Stats
eer_stats_list = []
# Calculating stats for classifiers
for geniune_scores_all, impostor_scores_all in scores:
    eer_stats = get_eer_stats(geniune_scores_all, impostor_scores_all)
    eer_stats_list.append(eer_stats)
# Generating report
generate_eer_report([*eer_stats_list], [*EER_model_names], os.path.join(output_folder, 'pyeer_report.csv'))
generate_eer_report([*eer_stats_list], [*EER_model_names], os.path.join(output_folder, 'pyeer_report.html'))
# Exporting error rates (Exporting FMR and FNMR to a CSV file)
# This is the DET curve, the ROC curve is a plot of FMR against 1 - FNMR
for eer_stats, model_name in zip(eer_stats_list, EER_model_names):
    out_file = os.path.join(output_folder, 'DET_{}.csv'.format(model_name))
    export_error_rates(eer_stats.fmr, eer_stats.fnmr, out_file)
# Plotting
plot_eer_stats([*eer_stats_list], [*EER_model_names], save_path=output_folder)